# The Colored Maps of Germany
The series of color-coded maps with the seven days incidence used for the video "germany_incidence_V4_300ms_1080p_music.mp4" is generated here.<br/>
The maps show all German counties forming the map of Germany, whereas the color of a county represents its seven days incidence as defined by the legend.<br/><br/>
Read the file ["Readme.ipynb"](Readme.ipynb) for more information.

## Modules
Needed to use non-Python functionalities already programmed by someone else.

In [1]:
# Used to cast the polygons into np.arrays and afterwards transpose them easily with the .T function
import numpy as np
import matplotlib.pyplot as plt    # to plot the counties
# Used to determine which part of the plot should be saved
from matplotlib.transforms import Bbox
# Used to underline text in plots
from matplotlib import rc
# Used to navigate the directories and check for files
import os
# backup directory of this file, because the working directory is changed to save files
plot_maps_for_video_directory = os.getcwd()

## Get the Data
Read the file ["Readme.ipynb"](Readme.ipynb) or check out the files ["get_data.ipynb"](get_data.ipynb) and ["get_geographical_data_of_german_counties.ipynb"](get_geographical_data_of_german_counties.ipynb) for more information about the data collection.

In [2]:
os.chdir(".."), os.chdir("collect_data")   # move to the cllect_data directory
%run get_data.ipynb
os.chdir(plot_maps_for_video_directory)    # return to original directory

Polished county data from file is ready to go!
Polished covid19 data from file is ready to go!


## Control
The list "list_of_dates_to_plot_incidences" contains all the dates for which a map of Germany with the counties colored corresponding to the seven days incidence of this date should be created.<br/>
Use "range(0, len(non_county_specific_data\['UTC'\]))" to plot maps for all dates. This very memory intensive task could get interrupted because the memory is full.

In [3]:
list_of_dates_to_plot_incidences = []

In [4]:
if len(list_of_dates_to_plot_incidences) > 0:
    # check if the needed directories are availlable - otherwise create them
    os.chdir(".."), os.chdir("..")   # move two directories up
    if not(os.path.isdir("results")): os.makedirs("results")
    os.chdir("results")    # move in the results directory
    if not(os.path.isdir("germany_incidence_V6")):
        os.makedirs("germany_incidence_V6")
    os.chdir(plot_maps_for_video_directory)    # return to original directory

### Defining the Color Spectrum
This part creates a color spectrum specifically for the German map of seven days incidences.<br/>
The seven days incidence of a county is directly interpreted as an index of the color spectrum. All incidences higher than the highest index are assigned the color of the last element in the spectrum.
<br/><br/>
As shown in the plot with the distribution of the number of seven days incidences, a seven days incidence between 0 and 50 is way more common amongst the counties than a seven days incidence between 300 and 350. Therefore the steps in the green spectrum are set to be bigger to make a bigger optical difference compared to the bigger numbers in the upper spectrum which only occur occasionally and differ more due to the exponential growth of a pandemic.
<br/><br/>
After multiple tries of plotting and tweaking the color spectrum, it more or less contains an exponential growth, which makes sense from an epidemiological perspective: If a county currently has 100 COVID-19 cases, it is way more likely that ten of them infect one person (in total ten) than that the only case in a county with currently one COVID-19 case infects ten more people. Therefore the jumps get statistically bigger the higher the seven days incidence grows.
<br/>
This is compensated by the step width of the color spectrum: the incidence must grow 10 times more in the highest part of the spectrum than in the lowest part of the spectrum to get the same numerical change of color due to the 10 times lower step width.
<br/><br/>
Or from another perspective: most of the time most of the counties are green and hard to distinguish, so the green color part is compressed to get more incidences in the red spectrum.

In [5]:
color_spectrum = list()
for color_step in np.arange(0.4,1,0.05):
    color_spectrum.append((0, color_step, 0))
for color_step in np.arange(0,1,0.02):
    color_spectrum.append((color_step, 1, 0))
for color_step in np.arange(0,1,0.01):
    color_spectrum.append((1, 1 - color_step, 0))
for color_step in np.arange(0,0.94,0.005):
    color_spectrum.append((1 - color_step, 0, 0))

In [6]:
def plot_map_of_germany_incidences(date_index):
    for AdmUnitID, county in covid19.items():
        if county['incidences'][date_index] >= len(color_spectrum):
            color = color_spectrum[-1]
        else:
            color = color_spectrum[int(county['incidences'][date_index])]

        for polygon in counties_geography[AdmUnitID]['geometry']:
            x, y = np.array(polygon).T
            plt.fill(x, y, color=color)
        # draw the lines around the counties
        for polygon in counties_geography[AdmUnitID]['raw_geometry']:
            x, y = np.array(polygon).T
            plt.plot(x, y, color="black")

In [7]:
def plot_legend_incidences():
    ax1 = plt.axes([0.01, 0.2, 0.06, 0.65], # postion of the legend: [left, bottom, width, height]
                   frameon=False)
    ax1.axes.get_xaxis().set_visible(False)

    ax1.yaxis.tick_right()
    plt.yticks([0,25,50,75,100,125,150,175,200,225,250,275,300,325,350],
               [0,25,50,75,100,125,150,175,200,225,250,275,300,325,">350"], fontsize=90)
    plt.ylabel('Seven-days-incidence', fontsize=100, rotation='vertical')

    for index in range(len(color_spectrum)):
        plt.fill([1, 2, 2, 1], 2*[1+index] + 2*[2+index], color=color_spectrum[index])
        if index % 25 == 0:    # create black lines besides the ticks
            plt.fill([1, 2, 2, 1], 2*[index] + 2*[1+index], color="black")

    # Add a black block at the end of the spectrum so that the highest tick (>350) has no ugly offset
    plt.fill([1, 2, 2, 1], 2*[len(color_spectrum)] + 2*[10+len(color_spectrum)], color="black")

In [8]:
def plot_histogram_incidences(date_index):
    # postion of the histogram: [left, bottom, width, height]
    plt.axes([0.96, 0.32, 0.55, 0.35], facecolor='lightgrey')
    plt.title(r"\textbf{Number of counties with given incidence}", fontsize=90, usetex=True, y=1.02, x=0.5)

    plt.xlabel("Incidence", fontsize=70)
    plt.xticks([num for num in range(0,351,50)], fontsize=70)
    plt.xlim(0, 350)

    plt.ylabel("Number of counties", fontsize=75)
    plt.yticks([num for num in range(10, 150, 10)], fontsize=70)
    plt.ylim(0, 150)

    _, _, bars = plt.hist([county['incidences'][date_index] for county in covid19.values()],
             bins=[num for num in range(0,351,25)], edgecolor="black")
    
    for bar in bars:
        plt.setp(bar, "facecolor", color_spectrum[int(bar.get_x())])

    plot_distribution_of_incidences_over_350(date_index)

In [9]:
def plot_distribution_of_incidences_over_350(date_index):
    # postion of the histogram: [left, bottom, width, height]
    ax = plt.axes([1.55, 0.32, 0.04, 0.35],facecolor='lightgrey')
    ax.axes.get_yaxis().set_visible(False)
    plt.ylim(0, 150)
    plt.xticks([375], [">350"], fontsize=70)
    plt.xlim(350, 400)
    
    _, _, bars = plt.hist([county['incidences'][date_index] for county in covid19.values()], bins=[350, 100000])
    
    plt.setp(bars[0], "facecolor", "black")

In [10]:
def display_text_incidences(date_index):
    # range of dates in the bottom left corner
    plt.text(0.5, -25, non_county_specific_data['UTC+7days'][date_index] + " to ", fontsize=80)
    plt.text(0.5, -39, non_county_specific_data['UTC+7days'][date_index + 7], fontsize=80)
    # title top right
    plt.text(17.5, 350, r'\textbf{\underline{Seven-days-incidence in Germany}}',
              fontweight=220, fontsize=160, usetex=True)
    # definition of seven days incidence top right
    plt.text(17.5, 330, "The number of COVID-19 cases in the last 7 days", fontsize=95)
    plt.text(17.5, 315, "of each German county scaled to 100.000 people.", fontsize=95)
    plt.text(17.5, 300, "Covers the period from " + non_county_specific_data['UTC'][0] +
             " to " + non_county_specific_data['UTC'][-1] + ".", fontsize=95)
    # data source and creater bottom right
    plt.text(17.5, 25, "Data from the COVID-19 Datenhub " +
             "referenced by the Robert-Koch-Institut (www.rki.de):", fontsize=55)
    plt.text(17.5, 15, 'npgeo-corona-npgeo-de.hub.arcgis.com ' +
             '(Used "RKI History" and "RKI Corona Landkreise")', fontsize=55)
    plt.text(17.5, 0, 'Created by Leander Bürkin (leander.buerkin@gmail.com),',fontsize=55)
    plt.text(17.5, -10, 'Laboratory for Simulation, Prof. Dr. Lars Pastewka, ' +
             'Albert-Ludwigs-Universität Freiburg', fontsize=55)
    plt.text(17.5, -20, 'https://github.com/leanderbuerkin/bachelor_project', fontsize=55)
    plt.text(17.5, -35, 'Music: Chad Crouch - Algorithms (original)', fontsize=55)

In [11]:
os.chdir(".."), os.chdir(".."), os.chdir("results")    # move in the results directory
for date_index in list_of_dates_to_plot_incidences:
    plt.figure(figsize = (50, 65), facecolor='darkgrey')
    ax = plt.axes(frameon=False)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    plot_map_of_germany_incidences(date_index)
    plot_histogram_incidences(date_index)
    plot_legend_incidences()
    display_text_incidences(date_index)
    plt.savefig("germany_incidence_V6/" + str(date_index) + '.png',
                facecolor="darkgrey", bbox_inches=Bbox([[-3.5,9],[84,57]]))
    plt.close('all')
os.chdir(plot_maps_for_video_directory)    # return to original directory